In [0]:
df = spark.sql("SELECT * FROM bronzelayer.agent WHERE agent.merge_flag = FALSE")
display(df)

1. Remove all rows where branch_Id not exist in branch table
2. Ensure all phone has valid 10 digits phone number
3. Replace all nul email with admin@admin.co
4. Add the merged_date with current timestamp 

Remove all rows where branch_Id does not exist in branch table

In [0]:
df_branch = spark.sql("Select * from bronzelayer.Branch")
df_result = spark.sql("SELECT agent.* FROM bronzelayer.agent INNER JOIN bronzelayer.branch on agent.branch_id = branch.branch_id WHERE agent.merge_flag = FALSE")
display(df_result)

Ensure all phone has valid 10 digits phone number

In [0]:
from pyspark.sql.functions import *

df_phone = df_result.filter(length(col("agent_phone") )==10)
display(df_phone)


Replace all null email with admin@admin.co

In [0]:
#df_email = df_phone.fillna({'agent_email':'admin@admin.co'})
df_email = df_phone.withColumn('agent_email', when(col('agent_email') == '', 'admin@admin.co').otherwise(col('agent_email')))
display(df_email)

Add the merged_date with current timestamp

In [0]:
# df_email.createOrReplaceTempView("clean_agent")
# spark.sql("merge into silverlayer.agent as T using clean_agent as S on t.agent_id = s.agent_id when matched THEN UPDATE SET t.agent_phone = s.agent_phone, t.agent_name = s.agent_name, t.agent_email = s.agent_email, t.agent_phone = s.agent_phone, t.branch_id = s.branch_id, t.create_timestamp = s.create_timestamp, t.merged_timestamp = current_timestamp() when not matched then INSERT (agent_phone, agent_name ,agent_email,agent_phone, branch_id, create_timestamp, merged_timestamp) values( s.agent_phone, s.agent_name,  s.agent_email, s.agent_phone, s.branch_id, s.create_timestamp, current_timestamp())")

df_merged_dt= df_email.withColumn("merged_timestamp", current_timestamp())
display(df_merged_dt)


writing into silverlayer db

In [0]:
df_merged_dt.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("silverlayer.Agent")

In [0]:
%sql
select * from silverlayer.agent

Update bronzelayer merge_flag =tru

In [0]:
spark.sql("UPDATE bronzelayer.agent SET merge_flag = True WHERE merge_flag = FALSE")